In [29]:
# Reload when code changed:
%load_ext autoreload
%autoreload 2
%pwd
import sys
import os
path = "../"
sys.path.append(path)
#os.path.abspath("../")
print(os.path.abspath(path))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
D:\Utveckling\git\ekostat_calculator


In [30]:
import os 
import core
import importlib
importlib.reload(core) 
try:
    logging.shutdown()
    importlib.reload(logging)
except:
    pass
import pandas as pd
import numpy as np
import json

from event_handler import EventHandler
print(core.__file__)
pd.__version__

..\core\__init__.py


'0.19.2'

In [26]:
import numpy as np

In [14]:


statn = np.array(list('ABCDEF')) 
wb = np.array(['a', 'a', 'a', 'b', 'b', 'b'])

In [15]:
include_statn = 'E'
exclude_statn = 'A'
include_wb = 'a'

In [16]:
b_inc_wb = (wb==include_wb)
b_inc_statn = (statn==include_statn)
b_exc_statn = ~(statn==exclude_statn) 

print('b_inc_wb:     ', b_inc_wb)
print('b_inc_statn:  ', b_inc_statn)
print('b_exc_statn:  ', b_exc_statn)

b_inc_wb:      [ True  True  True False False False]
b_inc_statn:   [False False False False  True False]
b_exc_statn:   [False  True  True  True  True  True]


In [17]:
print(b_inc_wb | b_inc_statn)

[ True  True  True False  True False]


In [20]:
print(b_inc_wb)
print(b_inc_statn)
print(b_exc_statn)
print((b_inc_wb | b_inc_statn) & b_exc_statn)

[ True  True  True False False False]
[False False False False  True False]
[False  True  True  True  True  True]
[False  True  True False  True False]


## Hypsograph

In [59]:
from core.mapping import Hypsograph 

file_path = 'D:/Utveckling/git/ekostat_calculator/resources/mappings/hypsografs_2017.txt'
h = Hypsograph(file_path)
wb = 'NO591045-111030'

In [60]:
h.get_total_area_of_water_body(wb)

4.6299999999999999

In [58]:
h.get_volume_fraction_below_depth()

,EUCD,Djup,Sum_Area(km2),Volym,acc_volume,frac_volume,frac_area
0,NO591045-111030,0,4.6300,0.0352,0.2556,1.000000,1.000000
1,NO591045-111030,1,3.6775,0.0311,0.2204,0.862285,0.794276
2,NO591045-111030,2,3.4600,0.0275,0.1893,0.740610,0.747300
3,NO591045-111030,3,3.2125,0.0242,0.1618,0.633020,0.693844
4,NO591045-111030,4,2.9900,0.0210,0.1376,0.538341,0.645788
